In [1]:
import JSSP
import gym
import time

In [ ]:

env_name = "JSSP-v0"
data = [[2,2],[2,1,0,5,1,1,10],[2,1,1,3,1,0,1]]
env = gym.make(env_name, env_data = data)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

In [ ]:
state = env.reset()
env.render(mode="human")
time.sleep(20)

# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         #env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))

In [12]:
import numpy as np


def initialize(instance_path):
    # input instance description for environment initialization
    instance_path = instance_path
    file_handle = open(instance_path, 'r')
    lines_list = file_handle.readlines()
    # first line consists of # of jobs in total, # of machines in total
    job_total, machine_total = [int(x) for x in lines_list[0].split()]
    # env_data = [[int(val) for val in line.split()] for line in lines_list[1:]]
    # read through each job description
    for job_index in range(len(lines_list) - 1):
        job_description = [int(val) for val in lines_list[job_index+1].split()]
        job_operation_map[job_index] = {}
        # initialize job_operation_map
            # ex. job_operation_map[1][2][3] = time it takes for 3rd machine to execute 2nd operation of 1st job
            # time = -1 iff the machine is not capable of executing the operation based on instance description
        for operation_index in range(job_description[0]):
            job_operation_map[job_index][operation_index] = np.negative(np.ones(machine_total))
        # populate job_description_map
        populate_job_description_map(job_description, job_index)

# Given an job description, populate the corresponding fields in job_description_map
def populate_job_description_map(job_description, job_index):
    # read job description from left to right
    description_pivot = 1
    operation_index = 0
    # operation_index
    while description_pivot < len(job_description):
        # operation_description length = 2 * # of machines capable of executing current operation at the pivot
        operation_description_end = 2 * job_description[description_pivot] + description_pivot
        # read the current description of operation
        description_pivot += 1
        while description_pivot <= operation_description_end:
            # Following the format of operation description:
            # machine index , time it takes for the current operation
            machine_index = job_description[description_pivot]
            operation_duration = job_description[description_pivot + 1]
            job_operation_map[job_index][operation_index][machine_index - 1] = operation_duration
            description_pivot += 2
        operation_index += 1

In [13]:

job_operation_map = {}
initialize('instance1.txt')
print(job_operation_map)

# first job second operation
print(job_operation_map[0][1])

{0: {0: array([10., 15., -1.]), 1: array([-1., 12., 18.])}, 1: {0: array([10., -1., 25.]), 1: array([25., 18., -1.]), 2: array([-1., 15., 25.])}}
[-1. 12. 18.]
